# Importing required packages

In [178]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Pulling raw data from web, using beautiful soup to organize

In [179]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_data= requests.get(wikipedia_link).text
data = BeautifulSoup(raw_data,'xml')


# Creating lists using 'for' iteration, 'find', and 'count'

In [180]:
#extracting the raw table inside that webpage
table = data.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []




for tr_cell in table.find_all('tr'):
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
            Postcode.append(Postcode_var)
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            Borough.append(Borough_var)
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            Neighbourhood.append(Neighbourhood_var)
        counter +=1
        

        

        
    
    
    

# Creating unique Postcode list

In [181]:

unique_postcode = set(Postcode)
print('Unique Postal codes:', len(unique_postcode))
Postcode_unique      = []
Borough_unique       = []
Neighbourhood_unique = []


for postcode_unique_element in unique_postcode:
    post_var = ''; bor_var = ''; neigh_var = ''; 
    for postcode_index, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            post_var = postcode_element;
            bor_var = Borough[postcode_index]
            if neigh_var == '': 
                neigh_var = Neighbourhood[postcode_index]
            else:
                neigh_var = neigh_var + ',' + Neighbourhood[postcode_index]
    Postcode_unique.append(post_var)
    Borough_unique.append(bor_var)
    Neighbourhood_unique.append(neigh_var)

Unique Postal codes: 180


# Creating dictionary and pandas dataframe

In [182]:

toronto_dict = {'Postcode':Postcode_unique, 'Borough':Borough_unique, 'Neighbourhood':Neighbourhood_unique}
df = pd.DataFrame.from_dict(toronto_dict)
df.to_csv('toronto_part1.csv')
df.shape

(180, 3)

# Refining dataframe

In [183]:
df = df.applymap(lambda x: x.replace('\n',''))
df = df[df['Borough'] != 'Not assigned']

In [184]:
df.Neighbourhood = df.Neighbourhood.apply(lambda x: x.replace('/',','))

In [185]:
df.shape

(103, 3)

In [186]:
df

,Postcode,Borough,Neighbourhood
0,M9N,York,Weston
4,M1W,Scarborough,"Steeles West , L'Amoreaux West"
8,M4E,East Toronto,The Beaches
9,M5G,Downtown Toronto,Central Bay Street
10,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place"
...,...,...,...
169,M6P,West Toronto,"High Park , The Junction South"
171,M7Y,East Toronto,Business reply mail Processing CentrE
173,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."
177,M3H,North York,"Bathurst Manor , Wilson Heights , Downsview North"


# Downloading Latitude and Longitude csv

In [187]:
import wget
wget.download('http://cocl.us/Geospatial_data')

'Geospatial_Coordinates (2).csv'

# Reading CSV

In [188]:
lat_lon = pd.read_csv('Geospatial_Coordinates.csv')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merging Latitude and Longitude with original DF

In [189]:
df = pd.merge(df,lat_lon, how = 'left', left_on ='Postcode', right_on = 'Postal Code', validate = '1:1')


In [190]:
df = df.drop('Postal Code', axis = 1)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9N,York,Weston,43.706876,-79.518188
1,M1W,Scarborough,"Steeles West , L'Amoreaux West",43.799525,-79.318389
2,M4E,East Toronto,The Beaches,43.676357,-79.293031
3,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
4,M6K,West Toronto,"Brockton , Parkdale Village , Exhibition Place",43.636847,-79.428191
